In [1]:
from pyraf import iraf
iraf.task(casleoccd="casleoccd/casleoccd.cl")
iraf.task(lacos_im="casleoccd/lacos_im.cl")
# iraf.casleoccd()

In [41]:
iraf.imred()
iraf.ccdred()
iraf.imutil()
iraf.noao()
iraf.digiphot()
iraf.apphot()
iraf.obsutil()
iraf.files()
# iraf.stsdas()
# iraf.lacos_im()

20240315
Calibradas1_Luis
ITFiles
add_casleoccd_task.py
base
casleoccd
data_hsh1
data_reduction.py
data_reduction_hsh.ipynb
data_reduction_iraf.py
login.cl
login.cl.OLD
pyraf
stsdas-site-manager-s-installation-guide-and-reference.pdf
uparm


In [53]:

# Set up IRAF display parameters
# iraf.set(stdimage='imt1024', StdoutGraf='no')

def casleoccd_hsh(band, night_path, sci_name = "sci", master_bias=None, master_flat=None, master_dark=None, cosmic_correction=True):
    instrumento = 'HSH-STL'  # Instrumento utilizado
    binning = 1  # Factor de binning (X,Y)
    redfoc = 'no'  # ¿Se usó reductor focal? (solo JS-Roper)
    ganron = 'no'  # ¿Calcula ganancia y ruido de lectura?
    sethead = 'yes'  # ¿Agrega/actualiza headers (EPADU y RDNOISE)?
    masterbias = 'no' if master_bias is None else master_bias  # ¿Genera master BIAS? (Zero)
    masterflat = 'no' if master_flat is None else master_flat  # ¿Genera master FLAT? (FlatX)
    masterdark = 'no' if master_dark is None else master_dark  # ¿Genera master DARK? (Dark)
    borracal = 'no'  # ¿Borra calibraciones originales?
    procesa = 'yes'  # ¿Procesa todas las imágenes?
    ovscinter = 'no'  # ¿Ajusta overscan en forma interactiva? (solo JS Roper y Tek)
    oscuridad = 'yes'  # ¿Corrige por dark? (solo HSH-STL)
    cosmicos = 'yes' if cosmic_correction else 'no'  # ¿Corrige por rayos cósmicos (lacos-im)?
    epadu = 1.0  # Valor de ganancia calculado (e-/adu)
    rdnoise = 0.0  # Valor de ruido de lectura calculado (e-)

    # Definir las rutas a los archivos de configuración
    base = 'base'  # Directorio base donde se encuentran los archivos de configuración
    listbias = os.path.join(night_path, "bias.list")
    listdark = os.path.join(night_path, "dark.list")
    listflat = os.path.join(night_path, f"flat{band}.list")
    listima = os.path.join(night_path, f"{sci_name}.list")
    # Llamada a la tarea de IRAF casleoccd
    iraf.casleoccd(instrumento=instrumento, binning=binning, redfoc=redfoc, ganron=ganron, sethead=sethead,
                   masterbias=masterbias, masterflat=masterflat, masterdark=masterdark, borracal=borracal,
                   procesa=procesa, ovscinter=ovscinter, oscuridad=oscuridad, cosmicos=cosmicos,
                   epadu=epadu, rdnoise=rdnoise, listbias=listbias, listflat=listflat,
                   listdark=listdark, listima=listima, flatcielo='no', flatHWP='no',
                   rfilJS=2, rfilHSH='ext', base=base, xcent=988, ycent=1072)
    

# def create_master_bias(bias_list, output='master_bias.fits'):
#     master_bias = 'yes' if output else 'no'
#     iraf.casleoccd(instrumento='', binning=1, redfoc='', ganron='', sethead='', masterbias=master_bias,
#                    masterflat='', masterdark='', borracal='', procesa='', ovscinter='', oscuridad='',
#                    cosmicos='', epadu=1.0, rdnoise=0.0, listbias=bias_list, listflat='', listdark='',
#                    listima='', flatcielo='', flatHWP='', rfilJS=0, rfilHSH='', base='', xcent=0, ycent=0,
#                    intmt='', subst='', output=output, StdoutGraf='no')

# def create_master_flat(flat_list, output='master_flat.fits'):
#     master_flat = 'yes' if output else 'no'
#     iraf.casleoccd(instrumento='', binning=1, redfoc='', ganron='', sethead='', masterbias='',
#                    masterflat=master_flat, masterdark='', borracal='', procesa='', ovscinter='', oscuridad='',
#                    cosmicos='', epadu=1.0, rdnoise=0.0, listbias='', listflat=flat_list, listdark='',
#                    listima='', flatcielo='', flatHWP='', rfilJS=0, rfilHSH='', base='', xcent=0, ycent=0,
#                    intmt='', subst='', output=output, StdoutGraf='no')

# def create_master_dark(dark_list, output='master_dark.fits'):
#     master_dark = 'yes' if output else 'no'
#     iraf.casleoccd(instrumento='', binning=1, redfoc='', ganron='', sethead='', masterbias='',
#                    masterflat='', masterdark=master_dark, borracal='', procesa='', ovscinter='', oscuridad='',
#                    cosmicos='', epadu=1.0, rdnoise=0.0, listbias='', listflat='', listdark=dark_list,
#                    listima='', flatcielo='', flatHWP='', rfilJS=0, rfilHSH='', base='', xcent=0, ycent=0,
#                    intmt='', subst='', output=output, StdoutGraf='no')

# def lacos_im(image_list):
#     # Llama a la tarea de IRAF lacos_im para la corrección de rayos cósmicos
#     iraf.lacos_im(images=image_list, output='cr_', sigclip=4.5, sigfrac=0.3, objlim=1.0, gain=1.0, readn=6.5,
#                   overwrite=True, interactive=False, verbose=True, StdoutGraf='no')


In [54]:
import os
import numpy as np
bands="iv"
nights_path = "data_hsh1"
nights = os.listdir(nights_path)
night_name = sorted(nights)[14]
images = [file for file in os.listdir(os.path.join(nights_path,night_name)) if file.endswith(".fit")] 
images_list = {}
objects = [image for image in images if ("dark" not in image) and ("bias" not in image) and ("flat" not in image)]
sci_name = "all"
images_list[sci_name] = objects
masters = ["dark", "bias"]
for band in bands:
    masters.append(f"flat{band}")
for master in masters:    
    images_list[master] = sorted([image for image in images if master in image])
for images in images_list.keys():
    np.savetxt(os.path.join(nights_path, night_name, f"{images}.list"), images_list[images], fmt="%s")
images_list.keys()

dict_keys(['all', 'dark', 'bias', 'flati', 'flatv'])

In [55]:
night_path = os.path.join(nights_path, night_name)
casleoccd_hsh(band, night_path, sci_name=sci_name, master_bias=None, master_flat=None, master_dark=None, cosmic_correction=False)

Killing IRAF task `ccdproc'


IrafError: Error running IRAF task ccdproc
IRAF task terminated abnormally
ERROR (827, "Cannot open image (Zero.fits)")


In [13]:
import os

nights = os.listdir("nights_images")
filt = "i"

# Se generan los master bias, flat y dark si no existen previamente
# if not os.path.isfile('master_bias.fits'):
#     bias_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "bias" in file]
#     create_master_bias(bias_list)

# if not os.path.isfile('master_flat.fits'):
#     flat_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if f"flat{filt}" in file]
#     create_master_flat(flat_list)

# if not os.path.isfile('master_dark.fits'):
#     dark_list = [file for file in os.listdir(f"nights_images/{nights[1]}") if "dark" in file]
#     create_master_dark(dark_list)
    

# Se generan las correcciones de rayos cósmicos si no existen previamente
if not os.path.isfile('cr_image1.fits'):
    lacos_im(['image1.fits'])
images_list = os.listdir(f"nights_images/{nights[2]}") 

casleoccd_hsh({'bias': 'master_bias.fits', 'flat': 'master_flat.fits', 'dark': 'master_dark.fits',
               'images': images_list}, cosmic_correction=False)


Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
ERROR! Session/line number was not unique in database. History logging moved to new session 458


In [18]:
os.listdir()

['.ipynb_checkpoints',
 'data_hsh1',
 '20240315',
 'stsdas-site-manager-s-installation-guide-and-reference.pdf',
 'add_casleoccd_task.py',
 'data_reduction_hsh.ipynb',
 'uparm',
 'login.cl.OLD',
 'base',
 'login.cl',
 'casleoccd',
 'data_reduction_iraf.py',
 'ITFiles',
 'pyraf',
 'Calibradas1_Luis',
 'data_reduction.py']

In [13]:
import os
nights = os.listdir("nights_images")
bias_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "bias" in file]
flat_i_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "flati" in file]
flat_v_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "flatv" in file]
dark_list = [file for file in os.listdir(f"nights_images/{nights[1]}") if "dark" in file]
os.listdir(f"nights_images/{nights[2]}") 

['CentaurusA_b0002.fit',
 'CentaurusA_b0007.fit',
 'CentaurusA_f0001.fit',
 'CentaurusA_f0006.fit',
 'CentaurusA_f0011.fit',
 'CentaurusA_f0012.fit',
 'CentaurusA_f0013.fit',
 'CentaurusA_f0014.fit',
 'CentaurusA_f0015.fit',
 'CentaurusA_f0016.fit',
 'CentaurusA_i0005.fit',
 'CentaurusA_i0010.fit',
 'CentaurusA_r0004.fit',
 'CentaurusA_r0009.fit',
 'CentaurusA_v0003.fit',
 'CentaurusA_v0008.fit']

In [16]:
import os

def listar_directorios(path, nivel=0):
    for elemento in os.listdir(path):
        ruta_completa = os.path.join(path, elemento)
        if os.path.isdir(ruta_completa):
            print("|   " * nivel + "|-- " + elemento + "/")
            if "2024" in elemento:
                continue
            listar_directorios(ruta_completa, nivel + 1)
        else:
            print("|   " * nivel + "|-- " + elemento)

if __name__ == "__main__":
    directorio_base =  '../Observations_HSH'
    if os.path.isdir(directorio_base):
        print("Árbol de directorios para:", directorio_base)
        listar_directorios(directorio_base)
    else:
        print("El directorio especificado no existe.")


('\xc3\x81rbol de directorios para:', '../Observations_HSH')
|-- stsdas
|-- uparm/
|   |-- clecl.par
|-- data_reduction.py
|-- redu.py
|-- subsets/
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- subsets_roper_r1
|-- Settings/
|   |-- ccdtek-HWP.dat
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- ccdtek.dat
|   |-- ccdroper_r1.dat
|   |-- ccdsbig_hsh_rint.dat
|   |-- ccdsbig_hsh_rext.dat
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- ccdroper_r2.dat
|   |-- subsets_roper_r1
|-- pyraf/
|   |-- clcache.v2/
|   |   |-- 0rwFKSevVndB54J0A+ivPg==
|   |   |-- V5A-jQWGB-i50zHVCLwi+Q==
|   |   |-- rA35OdKgMipLZ-qtNAUAbg==
|   |   |-- 1Vp-+GC6mmvzpgHCmNjA5g==
|   |   |-- aLo1NePVF8N8qDt5HC02xw==
|   |   |-- VU1ZH82P4r90jFXV0aAo6g==
|   |   |-- 7w5t990dmHyMvULHO7pi-A==
|   |   |-- urF7XXc0kB5SiMhRPc5zCg==
|   |   |-- vd5To6yC7aNdoCIY4Gah9A==
|   | 

In [17]:
listar_directorios(".", nivel=0)

|-- stsdas
|-- uparm/
|   |-- clecl.par
|-- data_reduction.py
|-- redu.py
|-- subsets/
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- subsets_roper_r1
|-- Settings/
|   |-- ccdtek-HWP.dat
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- ccdtek.dat
|   |-- ccdroper_r1.dat
|   |-- ccdsbig_hsh_rint.dat
|   |-- ccdsbig_hsh_rext.dat
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- ccdroper_r2.dat
|   |-- subsets_roper_r1
|-- pyraf/
|   |-- clcache.v2/
|   |   |-- 0rwFKSevVndB54J0A+ivPg==
|   |   |-- V5A-jQWGB-i50zHVCLwi+Q==
|   |   |-- rA35OdKgMipLZ-qtNAUAbg==
|   |   |-- 1Vp-+GC6mmvzpgHCmNjA5g==
|   |   |-- aLo1NePVF8N8qDt5HC02xw==
|   |   |-- VU1ZH82P4r90jFXV0aAo6g==
|   |   |-- 7w5t990dmHyMvULHO7pi-A==
|   |   |-- urF7XXc0kB5SiMhRPc5zCg==
|   |   |-- vd5To6yC7aNdoCIY4Gah9A==
|   |   |-- gHnB-9agvBoUT5ZpuQ52sg==
|   |   |-- 3lVIqYDK8prJ12m8Ge